In [ ]:
# default_exp days
# export
import sys
sys.path.append('..')

In [ ]:
#export
from seq_fastai.core import *
from seq_fastai.imports import *
from seq_fastai.torch_imports import *

In [ ]:
#export
from fastai2 import fastai2

In [ ]:
# export
class BaseDays(pd.DataFrame):
    def __init__(self, data, y_names, cat_var=None, con_var=None, cats={}, **kw):
        super(BaseDays, self).__init__(data, **kw)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.con_var, self.y_names = L(con_var), L(y_names)
            self.set_index("datetime", drop=False, inplace=True)

            self.cols = ["datetime"] + self.con_var
            self.cat_var = L(ifnone(cat_var, cats.keys()))
            self.cats =    cats

        if cats == {}:
            for cat_var in self.cat_var:
                self.cats[cat_var] = self[cat_var].iloc[0]
        else:
            for k, v in cats.items():
                self.loc[:, k] = v
        for c in self.cols + self.y_names:
            if c not in self.columns:
                print(f"Warning {c} not in {self.columns} of {self.__class__}")

    def copy(self):
        # https://stackoverflow.com/questions/17591104/in-pandas-can-i-deeply-copy-a-dataframe-including-its-index-and-column
        #         print('copy made ',self.__class__)
        return self.__class__(
            pd.DataFrame(self.values.copy(), self.index.copy(), self.columns.copy()),
            self.y_names.copy(),
            self.cat_var.copy(),
            self.con_var.copy(),
        )


In [ ]:
bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72),
                     'quater':np.arange(72)/4}),
         'target',
         con_var='quater'
        )
test_eq(bd.cats,{})
test_eq(bd.con_var,['quater'])

_bd = bd.copy()
_bd['half'] = _bd['quater']*2
del _bd['quater']
_bd.con_var=L('half')
test_eq(bd['quater'],np.arange(72)/4)


In [ ]:
bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72),
                     'one':[1]*72}),
         'target',
         cat_var='one'
        )
test_eq(bd.cats,{'one':1})
test_eq(bd.cat_var,['one'])

bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72)}),
         'target',
         cats={'one':1}
        )
test_eq(bd.cats, {'one':1})
test_eq(bd.cat_var, ['one'])


In [ ]:
# export
class Days(BaseDays):
    def show(self, plot=True, **kwargs):

        title = f"{self.cats}  from {str(min(self.df.loc[:,'datetime']))} to {str(max(self.df.loc[:,'datetime']))} #{self.df.shape[0]}"
        print(self.df.columns)
        if plot:
            plot_days(
                self.df,
                left=self.con_var,
                right=self.y_names + ["pred"],
                title=title,
                **kwargs,
            )
        else:
            return title + "\n" + str(self.df.head(1))

    @classmethod
    def from_ndays(cls, nday):
        cats = {} if not hasattr(nday, "cats") else nday.cats
        return cls(nday, nday.y_names, nday.cat_var, nday.con_var, cats=cats)


In [ ]:
# TODO add tests

In [ ]:
# export
class NotDays(BaseDays):
    @classmethod
    def from_days(cls, day):
        cats = {} if not hasattr(day, "cats") else day.cats
        return cls(day, day.y_names, day.cat_var, day.con_var, cats=cats)


In [ ]:
# TODO add tests

In [ ]:
# export
def plot_days(data, ax=None, left=None, right=None, figsize = None, title=None, spacing=.1,  **kwargs):
    from pandas import plotting
    figsize = ifnone(figsize,(5,5))
    if ax is None: _,ax = plt.subplots(figsize=figsize)           
    colors = getattr(getattr(plotting, '_matplotlib').style, '_get_standard_colors')(num_colors=len(left)+len(right))
    for c in left+right:
        if c not in data:
            print(f'Warning{c} not in {data.columns}')
    # First axis
    ax = data.loc[:, left].plot(label=left, color=colors[:len(left)],ax = ax)
    ax.set_ylabel(ylabel=left)
    lines, labels = ax.get_legend_handles_labels()

    for n in range(len(right)):
        # Multiple y-axes 
        if right[n] in data.columns:
            
            ax_new = ax.twinx()
            ax_new.spines['right'].set_position(('axes', 1 + spacing * (n - 1)))
            data.loc[:, right[n]].plot(ax=ax_new, label=right[n], color=colors[len(left)+n % len(colors)],)
            ax_new.set_ylabel(ylabel=right[n])

            # Proper legend position
            line, label = ax_new.get_legend_handles_labels()
            lines += line
            labels += label

    ax.legend(lines, labels, loc=0)
    if title is not None: ax.set_title(title)
    return ax

In [ ]:
# export
def to_elapsed(s):
    return s.astype(np.int64) // 10 ** 9

def remove_last(df,start, end, sz):
    old_start = copy(start)
    start = end - sz + pd.Timedelta(hours = 1)
    return df[start:end], old_start, start
    
def get_not_interval(df, interval, shift = 1):
    df["delta"] = abs(
        (to_elapsed(df["datetime"]) - to_elapsed(df["datetime"].shift(shift)))
    )
    #     df["delta"] = df['delta'].dt.seconds
    not_hour = df.index[df["delta"] != interval].tolist()
    del df["delta"]
    return not_hour
    

In [ ]:
# export
def make_interval(
    df: pd.DataFrame,
    sz='96H',
    interval=3600,
    max_splits=35,
    callback_error=None,
) -> pd.DataFrame:
    """Will check if `df.datetime` has interval of `interval` in seconds. 
    
    if not will make it happen and return a list where this is done.
    """
    df.index = df.datetime
    df = df.sort_index() 
    starts = get_not_interval(df, interval, 1)
    ends = get_not_interval(df, interval, -1)
    dfs = []
    for start, end in zip(starts,ends):  
        dt_range = pd.date_range(start,end,freq=sz)
        dfs += [df.loc[i:i+pd.Timedelta(sz),:] for i in dt_range[:-1] ]   
        if (end -dt_range[-1]).total_seconds()/3600 >3:
            dfs += [df.loc[dt_range[-1]:end,:]]
    return dfs

In [ ]:
# TODO add tests

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

ModuleNotFoundError: No module named 'nbdev'